In [4]:
import torch
import torchaudio
import torchaudio.functional as F

print(torch.__version__)
print(torchaudio.__version__)

import matplotlib.pyplot as plt
import numpy as np
import librosa
import soundfile as sf
import pandas as pd
from tqdm import tqdm
import os
import random

c:\Users\user\anaconda3\envs\asvspoof2021\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\anaconda3\envs\asvspoof2021\lib\site-packages\torchaudio\extension\extension.py:14: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


1.6.0
0.6.0


In [2]:
from IPython.display import Audio

# Making Noise

In [16]:
noise, sr = librosa.load('data/unlabeled_data/GJNSVOZO.ogg')
noise = noise[22050*2:]
noise = np.append(noise, noise, axis=0)
noise = np.append(noise, noise, axis=0)
noise = noise + noise
noise.shape

(264600,)

In [ ]:
Audio(noise, rate=22050)

In [17]:
sf.write('GJNSVOZO_noise.ogg', noise, sr, format='OGG')

: 

# Adding Noise

In [4]:
# prompt: data_path = '/content/drive/MyDrive/Colab_Notebooks/DACON/spoof/data/train' 에서의 파일 목록 출력

import os

data_path = '../data/train'
noise_path = '../data/cut_noise'

# Get the list of files in the directory
filelist = os.listdir(data_path)
noiselist = os.listdir(noise_path)

print(len(filelist), len(noiselist))


55438 19800


In [4]:
len(filelist)

55438

In [5]:
for n in tqdm(noiselist):
    filelist.remove(n)

100%|██████████| 51704/51704 [00:00<00:00, 148738.96it/s]


In [6]:
from tqdm import tqdm
import pandas as pd

for idx in tqdm(range(len(filelist))):
  speech, sr = librosa.load('data/train/'+filelist[idx])
  noise, _ = librosa.load('GJNSVOZO_noise.ogg')
  file_path = 'noise/'+filelist[idx]
  if speech.shape[0] > noise.shape[0]:
      pad_width = speech.shape[0] - noise.shape[0]
      zero_array = np.zeros(pad_width)
      noise = np.append(noise, zero_array, axis=0)
      # print(noise.shape)
  else:
      noise = noise[:speech.shape[0]]
  noisy_speech = speech + noise

  sf.write(file_path, noisy_speech, sr, format='OGG')

100%|██████████| 3734/3734 [03:06<00:00, 20.04it/s]


In [8]:
data_path = 'data/train'
filelist = os.listdir(data_path)
noise_files = [os.path.join('data/noise_for_aug', f) for f in os.listdir('data/noise_for_aug')]
noise_files

['data/noise_for_aug\\noise_ACHLOQRM.ogg',
 'data/noise_for_aug\\noise_AHHFPCPB.ogg',
 'data/noise_for_aug\\noise_AJHEAGQS.ogg',
 'data/noise_for_aug\\noise_BDCDLPPL.ogg',
 'data/noise_for_aug\\noise_BDHSWTSL.ogg',
 'data/noise_for_aug\\noise_BKIRLZMS.ogg',
 'data/noise_for_aug\\noise_BRAAQRBI.ogg',
 'data/noise_for_aug\\noise_BRWTPRLV.ogg',
 'data/noise_for_aug\\noise_BYJDOELP.ogg',
 'data/noise_for_aug\\noise_CAAGJRBF.ogg',
 'data/noise_for_aug\\noise_CCGZXAOC.ogg',
 'data/noise_for_aug\\noise_CNZIDPHH.ogg',
 'data/noise_for_aug\\noise_CPJUQQOI.ogg']

In [15]:
def calculate_rms(signal):
    return np.sqrt(np.mean(signal**2))

def add_noise(org_csv, noise_csv, save_path):
    org_files = pd.read_csv(org_csv)['path'].tolist()
    org_fs = pd.read_csv(org_csv)['id'].to_list()
    noise_files = pd.read_csv(noise_csv)['path'].tolist()

    for idx in tqdm(range(len(org_files))):
        audio, sr = librosa.load('data/'+org_files[idx][1:])
        noise_file = random.choice(noise_files)
        noise, noise_sr = librosa.load('data/cut_noise/'+noise_file)

        if len(audio) < sr*5:
            padding = np.zeros((sr*5)- len(audio))
            audio = np.concatenate((audio, padding))

        audio = audio[:sr*5]
        audio = audio * 0.2
        noise = noise[:sr*5]
        # print(audio.shape)
        
        rms_signal = calculate_rms(audio)

        snr_db = 0.1
        snr_linear = 10 ** (snr_db / 20)
        rms_noise = rms_signal / snr_linear
        current_rms_noise = calculate_rms(noise)
        noise_scaling_factor = rms_noise / current_rms_noise
        adjusted_noise = noise * noise_scaling_factor
        noisy_audio = audio + adjusted_noise
        file_path = save_path + org_fs[idx] + '.ogg'
        
        sf.write(file_path, noisy_audio, sr, format='OGG')

In [16]:
org_path = 'data/padded_combined_train_for_noise.csv'
noise_path = 'data/noise_for_add.csv'
save_path = 'noise/'

add_noise(org_path, noise_path, save_path)

 50%|████▉     | 14901/30000 [20:57<21:14, 11.85it/s]  


KeyboardInterrupt: 

In [10]:
22050*5

110250

In [29]:
import os
import shutil

def copy_excluding_list(source_dir, target_dir, exclude_list):
    """
    source_dir에 있는 파일 중 exclude_list에 없는 파일만 target_dir로 복사합니다.

    Parameters:
    source_dir (str): 원본 폴더 경로
    target_dir (str): 복사할 대상 폴더 경로
    exclude_list (list): 복사하지 않을 파일 목록
    """
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    
    for file_name in tqdm(os.listdir(source_dir)):
        if file_name not in exclude_list:
            full_file_name = os.path.join(source_dir, file_name)
            if os.path.isfile(full_file_name):
                shutil.copy(full_file_name, target_dir)
                # print(f"Copied: {file_name}")

# 사용 예시
source_directory = 'data/padded_combined_train'  # 원본 폴더 경로
target_directory = 'comb_noise_added'  # 대상 폴더 경로
files_to_exclude = os.listdir('comb_noise_added/')  # 제외할 파일 목록

copy_excluding_list(source_directory, target_directory, files_to_exclude)


100%|██████████| 60000/60000 [02:13<00:00, 450.89it/s]


In [30]:
len(os.listdir('comb_noise_added/'))

60000

In [18]:
len(os.listdir('data/padded_combined_train_10000/'))

10000

In [21]:
import shutil

In [29]:
shutil.move('data/padded_combined_train/mixed_combined_30000.ogg', 'mixed_combined_30000.ogg')

'mixed_combined_30000.ogg'

In [28]:
len(os.listdir('padded/'))

10000